In [13]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/lincolnzjx/anaconda3/envs/Py36/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['gamma', 'exp', 'beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
import pandas as pd

In [15]:
cd ../

/home/lincolnzjx/Desktop


In [16]:
pwd

'/home/lincolnzjx/Desktop'

In [17]:
def main(cuda, exp, alpha, beta, gamma, selected_layer, selected_filter,
         class_index, num_class, re_size, dataset, backbone, img_index, mode,
         inter, defense):
    resume = "037-0"
    server = 'desktop'
    n_epochs = 10000
    seed = -1
    eval_frequency = 50
    delta = 0
    batch_size = 1 if mode else num_class
    lr = 1
    
    !CUDA_VISIBLE_DEVICES=$cuda python -u -W ignore src/trainer.py \
        --experiment_index $exp\
        --alpha=$alpha \
        --gamma=$gamma \
        --beta=$beta \
        --selected_layer=$selected_layer \
        --selected_filter=$selected_filter \
        --num_class=$num_class \
        --class_index=$class_index \
        --server $server\
        --dataset $dataset\
        --n_epochs $n_epochs \
        --eval_frequency=$eval_frequency \
        --re_size $re_size \
        --backbone=$backbone \
        --optimizer=SGD \
        --learning_rate $lr \
        --delta=$delta \
        --resume $resume \
        --mode=keep \
        --inter=False \
        --regularization=L1 \
        --seed $seed \
        --img_index $img_index\
        --batch_size $batch_size\
        --defense $defense

# Read From excel

In [18]:
excel_path = "./src/exps.xlsx"
sheet = "ImageNet"
excepts = []

# ALL YOU NEED TO SET HERE!!

In [19]:
exp_index = "x091810"

# Main

In [20]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, dtype={"exp": str, 
                                          "Done": str,
                                          "selected_layer": int,
                                          "selected_filter": int, 
                                          "Alpha": int,
                                          "Beta": int,
                                          "Gamma": int,
                                          "cuda": int,
                                          "class_index": int,
                                          "num_class": int,
                                          "re_size": int,
                                          "dataset": str,
                                          "backbone": str, 
                                          "mode":str,
                                          "defensed": str}).values

for data in excel:
    exp, done, selected_layer, selected_filter, alpha, beta, gamma, cuda, \
        class_index, num_class, re_size, dataset, backbone, mode, inter, defensed = data
    if exp != exp_index:
        continue
        
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            for img_index in range(30):
                if img_index == 0:
                    print("=> RUN on ", cuda, exp, selected_layer, selected_filter, 
                          class_index, "img_index ", img_index)
                    main(cuda, exp, alpha, beta, gamma, selected_layer, 
                         selected_filter, class_index, num_class, re_size, dataset,
                         backbone, img_index, mode, inter, defensed)

FileNotFoundError: [Errno 2] No such file or directory: './src/exps.xlsx'

# Visualization

In [ ]:
import pickle

In [ ]:
statistic_path = "/home/lincolnzjx/Desktop/Interpretation/saved/generated/091810/n07747607_10061_statistics.pl"
# statistic_path = "/home/lincolnzjx/Desktop/Interpretation/saved/generated/091810/n07747607_1021_statistics.pl"

In [ ]:
with open(statistic_path, "rb") as handle:
    statistic = pickle.load(handle)

In [ ]:
epoch = statistic["epoch"]
loss = statistic["losses"]
selected_filter_loss = statistic["selected_filter_loss"]
rest_filter_loss = statistic["rest_fileter_loss"]
rest_filter_loss_interact = statistic["rest_filter_loss_interact"]
regularization_loss = statistic["regularization_loss"]
lr = statistic["lr"]
best = statistic["best"][0]

metric_statistic = statistic["metric_statistic"]
rmses_mean = [ms["rmses_mean"] for ms in metric_statistic]
ssim_sel_mean = [ms["ssim_sel_mean"] for ms in metric_statistic]
ssim_res_mean = [ms["ssim_res_mean"] for ms in metric_statistic]

print("sf: min:{:.3f} max:{:.3f}".format(min(selected_filter_loss), max(selected_filter_loss)))
print("rf: min:{:.3f} max:{:.3f}".format(min(rest_filter_loss), max(rest_filter_loss)))
print("rfi: min:{:.3f} max:{:.3f}".format(min(rest_filter_loss_interact), max(rest_filter_loss_interact)))
print("rg: min:{:.3f} max:{:.3f}".format(min(regularization_loss), max(regularization_loss)))
print("rmse: min:{:.3f} max:{:.3f}".format(min(rmses_mean), max(rmses_mean)))
print("ssim_sel: min:{:.3f}, max:{:.3f}".format(min(ssim_sel_mean), max(ssim_sel_mean)))
print("ssim_res: min:{:.3f}, max:{:.3f}".format(min(ssim_res_mean), max(ssim_res_mean)))
print("-"*50)
for key, name in best.items():
    if key != "processed_images":
        print(key, name)


fig = plt.figure(figsize=(3, 21))
fig.add_axes(sharex=True)
# plt.xlim((0, max(epoch)+1))

ax = plt.subplot(711)
plt.plot(epoch, selected_filter_loss, label="selected_filter_loss")
plt.legend(loc='best')

plt.subplot(712)
plt.plot(epoch, rest_filter_loss, label="rest_filter_loss")
plt.plot(epoch, rest_filter_loss_interact, label="rest_filter_loss_interact")
plt.legend(loc='best')

plt.subplot(713)
plt.plot(epoch, regularization_loss, label="regularization")
plt.legend(loc='best')

plt.subplot(714)
plt.plot(epoch, loss, label="train_loss")
plt.legend(loc='best')

plt.subplot(715)
plt.plot(epoch, lr, label="lr")
plt.legend(loc='best')

plt.subplot(716)
plt.plot(epoch, rmses_mean, label="rmse")
plt.legend(loc='best')

plt.subplot(717)
plt.plot(epoch, ssim_sel_mean, label="ssim_sel")
plt.plot(epoch, ssim_res_mean, label="ssim_res")
plt.legend(loc='best')

plt.suptitle('Training Process')
plt.show()